# stata

> Simple wrappers for `pystata`/`sfi` functionality

`sfi` is [Stata's python API](https://www.stata.com/python/api17/index.html), originally intended for interacting with Stata from python *within Stata*. As such, it can only be imported with Stata running.
`pystata.stata.run` [enables running Stata code from python](https://www.stata.com/python/pystata/stata.html#pystata.stata.run).

For one thing, using wrappers removes the need to constantly import `pystata` and/or `sfi` within functions elsewhere to avoid running afoul of CI tests in an environment without Stata installed.

In [ ]:
#| default_exp stata
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.config import launch_stata
from nbstata.misc_utils import DivertedPrints

In [ ]:
from textwrap import dedent

https://www.stata.com/python/pystata/stata.html#pystata.stata.run

In [ ]:
#| export
def run_direct(cmd, quietly=False, echo=False, inline=True):
    import pystata
    if quietly:
        with DivertedPrints() as diverted: # to prevent blank line output, as with `program define`
            out = pystata.stata.run(cmd, quietly, echo, inline)
            prints = diverted.getvalue()
        for line in prints.splitlines():
            if line.strip():
                print(line)
        return out
    else:
        return pystata.stata.run(cmd, quietly, echo, inline)

In [ ]:
#| hide
#| eval: False
launch_stata(splash=False)
import pystata
prog_code = "program define _temp_prog \n disp 1 \n end"
pystata.stata.run(prog_code, quietly=True)

In [ ]:
#| hide
#| eval: False
pystata.stata.run("capture program drop _temp_prog", quietly=True)
run_direct(prog_code, quietly=True)

In [ ]:
#| hide
#| eval: False
code = dedent('''\
    python:
    print("hello")
    end
    ''')
run_direct(code, quietly=True)

hello


In [ ]:
#| hide
#| eval: False
code = dedent('''\
    python:
    do_nothin = 3
    end
    ''')
run_direct(code, quietly=True)

[https://www.stata.com/python/api16/Macro.html#sfi.Macro.getLocal](https://www.stata.com/python/api16/Macro.html#sfi.Macro.getLocal)

In [ ]:
#| export
def get_local(name):
    import sfi
    return sfi.Macro.getLocal(name)

https://www.stata.com/python/api16/Macro.html#sfi.Macro.setLocal

In [ ]:
#| export
def set_local(name, value):
    import sfi
    return sfi.Macro.setLocal(name, value)

https://www.stata.com/python/api16/Macro.html#sfi.Macro.getGlobal

In [ ]:
#| export
def get_global(name):
    import sfi
    return sfi.Macro.getGlobal(name)

https://www.stata.com/python/api16/SFIToolkit.html#sfi.SFIToolkit.formatValue

In [ ]:
#| export
def stata_formatted(value, s_format):
    import sfi
    return sfi.SFIToolkit.formatValue(value, s_format)

`variable_names` uses 'getVarCount' and 'getVarName': https://www.stata.com/python/api16/Data.html 

In [ ]:
#| export
def variable_names():
    from sfi import Data
    return [Data.getVarName(i) for i in range(Data.getVarCount())]

In [ ]:
#| eval: false
run_direct("gen var1 = 1", quietly=True)
variable_names()

['var1']

https://www.stata.com/python/api16/Data.html#sfi.Data.dropVar

In [ ]:
#| export
def drop_var(name):
    import sfi
    sfi.Data.dropVar(name)

In [ ]:
#| eval: False
drop_var('var1')
variable_names()

[]

https://www.stata.com/python/api16/Data.html#sfi.Data.getObsTotal

In [ ]:
#| export
def obs_count():
    """Count the number of observations"""
    import sfi
    return sfi.Data.getObsTotal()

In [ ]:
#| eval: false
obs_count()

0

https://www.stata.com/python/api16/SFIToolkit.html#sfi.SFIToolkit.getWorkingDir

In [ ]:
#| export
def pwd():
    from sfi import SFIToolkit
    return SFIToolkit.getWorkingDir()

In [ ]:
#| eval: False
pwd()[-12:]

'\\nbstata\\nbs'

The following function is not currently used and is likely made unnecessary by `sfi.SFIToolkit.macroExpand`.

In [ ]:
#| export
def resolve_macro(macro):
    macro = macro.strip()
    if macro.startswith("`") and macro.endswith("'"):
        macro = get_local(macro[1:-1])
    elif macro.startswith("$_"):
        macro = get_local(macro[2:])
    elif macro.startswith("${") and macro.endswith("}"):
        macro = get_global(macro[2:-1])
    elif macro.startswith("$"):
        macro = get_global(macro[1:])
    return macro

In [ ]:
#| eval: false
run_direct('global test = "hello world"')
resolve_macro("$test")

'hello world'

In [ ]:
#| hide
#| eval: false
resolve_macro("${test}")

'hello world'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()